# **Water Potability FNN MODEL**

In [1]:
# ============================================
# Deep FNN + FLOPS & GFLOPS Calculation
# Water Potability Dataset
# ============================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# ------------------------------------------------
# 1) Load Dataset
# ------------------------------------------------
df = pd.read_csv(r"D:\college\sem-8\dataset\water_potability.csv")
df = df.fillna(df.mean())  # handle missing

X = df.drop("Potability", axis=1)
y = df["Potability"]

# ------------------------------------------------
# 2) Train/Test Split + Scaling
# ------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

input_dim = X_train.shape[1]


# ------------------------------------------------
# 3) Build DEEP FNN Model (INCREASED DENSE LAYERS)
# ------------------------------------------------
model = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),
    Dropout(0.2),

    Dense(64, activation='relu'),
    Dropout(0.2),

    Dense(32, activation='relu'),
    Dropout(0.2),

    Dense(16, activation='relu'),

    Dense(8, activation='relu'),

    Dense(1, activation='sigmoid')   # Binary output
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


# ------------------------------------------------
# 4) FLOPS Calculation
# ------------------------------------------------

def dense_layer_flops(input_units, output_units):
    # multiply + add = 2 FLOPS
    return input_units * output_units * 2

def calculate_fnn_flops(model):
    total_flops = 0
    layer_details = []

    prev_units = input_dim

    for layer in model.layers:
        if isinstance(layer, Dense):
            units = layer.units
            flops = dense_layer_flops(prev_units, units)

            layer_details.append((layer.name, prev_units, units, flops))
            total_flops += flops

            prev_units = units

    return total_flops, layer_details

total_flops, layer_details = calculate_fnn_flops(model)

print("\n================ FLOPS DETAILS ================")
for name, inp, out, fl in layer_details:
    print(f"{name}: {inp} → {out}  FLOPS = {fl:,}")

print("\nTotal FLOPS =", f"{total_flops:,}")
print("Total GFLOPS =", total_flops / 1e9)


# ------------------------------------------------
# 5) Train Model
# ------------------------------------------------
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1
)

# ------------------------------------------------
# 6) Evaluate
# ------------------------------------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {acc * 100:.2f}%")

# ------------------------------------------------
# 7) Save Model
# ------------------------------------------------
model.save("deep_water_potability_fnn_model.h5")
print("\nModel saved as deep_water_potability_fnn_model.h5")


c:\Users\Ishan\Documents\sample_p1\env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,289 (48.00 KB)

 Trainable params: 12,289 (48.00 KB)

 Non-trainable params: 0 (0.00 B)


================ FLOPS DETAILS ================
dense: 9 → 128  FLOPS = 2,304
dense_1: 128 → 64  FLOPS = 16,384
dense_2: 64 → 32  FLOPS = 4,096
dense_3: 32 → 16  FLOPS = 1,024
dense_4: 16 → 8  FLOPS = 256
dense_5: 8 → 1  FLOPS = 16

Total FLOPS = 24,080
Total GFLOPS = 2.408e-05
Epoch 1/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6020 - loss: 0.6809 - val_accuracy: 0.6050 - val_loss: 0.6689
Epoch 2/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6109 - loss: 0.6681 - val_accuracy: 0.6050 - val_loss: 0.6611
Epoch 3/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5883 - loss: 0.6681 - val_accuracy: 0.6050 - val_loss: 0.6504
Epoch 4/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5998 - loss: 0.6506 - val_accuracy: 0.6298 - val_loss: 0.6430
Epoch 5/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6337 - loss: 0.6382 - val_accuracy: 0.6813 - val_loss: 0.6343
Epoch 6/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6577 - loss: 0.6157 - val_


Test Accuracy: 67.07%

Model saved as deep_water_potability_fnn_model.h5


# **CNN MODEL FOR FLOWER DATASET**

In [3]:
# =====================================================
# CNN for Flower Photos Dataset
# Dataset: flower_photos (5 classes)
# =====================================================

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os

# -----------------------------------------------------
# 1) Set dataset path
# -----------------------------------------------------
dataset_path = r"D:\college\sem-8\dataset\flower_photos"   # folder that contains subfolders

# -----------------------------------------------------
# 2) Create Train/Validation Splits
# -----------------------------------------------------
IMG_SIZE = (180, 180)
BATCH_SIZE = 32

train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=42,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=42,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

class_names = train_ds.class_names
print("Classes:", class_names)

# -----------------------------------------------------
# 3) Prefetch for speed
# -----------------------------------------------------
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# -----------------------------------------------------
# 4) Build CNN Model
# -----------------------------------------------------
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(180, 180, 3)),
    MaxPooling2D(),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(),    
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(len(class_names), activation='softmax')   # 5 classes
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------------------------------
# 5) Train
# -----------------------------------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

# -----------------------------------------------------
# 6) Evaluate
# -----------------------------------------------------
loss, acc = model.evaluate(val_ds)
print(f"\nValidation Accuracy: {acc*100:.2f}%")

# -----------------------------------------------------
# 7) Save Model
# -----------------------------------------------------
model.save("flower_cnn_model.h5")
print("Saved as: flower_cnn_model.h5")


Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
Classes: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 18, 18, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 7, 7, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 536,645 (2.05 MB)

 Trainable params: 536,645 (2.05 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 50s 515ms/step - accuracy: 0.2760 - loss: 5.3989 - val_accuracy: 0.4183 - val_loss: 1.2863
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 44s 477ms/step - accuracy: 0.4541 - loss: 1.3062 - val_accuracy: 0.5940 - val_loss: 1.0278
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 45s 486ms/step - accuracy: 0.5616 - loss: 1.0941 - val_accuracy: 0.6621 - val_loss: 0.8859
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 46s 495ms/step - accuracy: 0.6108 - loss: 0.9827 - val_accuracy: 0.6689 - val_loss: 0.8719
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 51s 557ms/step - accuracy: 0.6629 - loss: 0.8738 - val_accuracy: 0.6798 - val_loss: 0.8717
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 137ms/step - accuracy: 0.6737 - loss: 0.8675



Validation Accuracy: 67.98%
Saved as: flower_cnn_model.h5


# **CNN MOdel FOR Trash Mangement**

In [5]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout

# ---- 1. Paths and parameters ----
DATASET_DIR = r"D:\college\sem-8\dataset\DATASET"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5
AUTOTUNE = tf.data.AUTOTUNE

# ---- 2. Build training and validation datasets ----
train_dir = os.path.join(DATASET_DIR, "train")

train_ds = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=42
)

val_ds = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=42
)

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# ---- 3. Define CNN Model ----
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(224,224,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Conv2D(128, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))

# Final layer for binary classification
model.add(Dense(1, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

# ---- 4. Train ----
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

# ---- 5. Save model ----
model.save("waste_classification_cnn.h5")
print("Model saved as waste_classification_cnn.h5")


Found 22564 files belonging to 2 classes.
Using 18052 files for training.
Found 22564 files belonging to 2 classes.
Using 4512 files for validation.


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_13 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 222, 222, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 109, 109, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 52, 52, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,261,185 (84.92 MB)

 Trainable params: 22,261,185 (84.92 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
121/565 ━━━━━━━━━━━━━━━━━━━━ 10:36 1s/step - accuracy: 0.5560 - loss: 68.3838

: 

# **Datasets Reduction**

In [7]:
import os
import shutil
import random

source_dir = r"D:\college\sem-8\dataset\Garbage classification\Garbage classification"
target_dir = r"D:\college\sem-8\dataset\reduced_garbage_dataset"
percent = 0.3   # keep 30%

os.makedirs(target_dir, exist_ok=True)

for cls in os.listdir(source_dir):
    cls_path = os.path.join(source_dir, cls)
    if not os.path.isdir(cls_path):
        continue

    os.makedirs(os.path.join(target_dir, cls), exist_ok=True)

    images = os.listdir(cls_path)
    random.shuffle(images)

    num_keep = int(len(images) * percent)
    selected = images[:num_keep]

    for img in selected:
        shutil.copy(os.path.join(cls_path, img),
                    os.path.join(target_dir, cls, img))

print("Reduced dataset created at:", target_dir)


Reduced dataset created at: D:\college\sem-8\dataset\reduced_garbage_dataset


DATA CLEANING

In [1]:
import os
import shutil
from PIL import Image

def clean_and_save_dataset(input_folder, output_folder):
    removed = 0
    saved = 0

    # create output folder
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder)

    for root, dirs, files in os.walk(input_folder):
        for file in files:
            in_path = os.path.join(root, file)

            # skip non-image files
            if not file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                continue  

            # output path
            rel_path = os.path.relpath(root, input_folder)
            out_dir = os.path.join(output_folder, rel_path)
            os.makedirs(out_dir, exist_ok=True)
            out_path = os.path.join(out_dir, file)

            try:
                img = Image.open(in_path)

                # Re-open to load pixel data
                img = Image.open(in_path)

                # Convert invalid modes → RGB
                if img.mode not in ['RGB', 'RGBA', 'L']:
                    print("Fixing mode:", img.mode, "-> RGB", in_path)
                    img = img.convert("RGB")

                # Ensure 3 channels (RGB)
                if img.mode == "L":
                    print("Converting 1-channel to RGB:", in_path)
                    img = img.convert("RGB")

                if img.mode == "RGBA":
                    print("Converting RGBA to RGB:", in_path)
                    img = img.convert("RGB")

                # Save to new folder
                img.save(out_path)
                saved += 1

            except Exception as e:
                print("Skipping corrupt file:", in_path)
                removed += 1

    print("✔ Finished cleaning.")
    print("✔ Saved images:", saved)
    print("✘ Skipped corrupted images:", removed)

input_folder = r"D:\college\sem-8\dataset\PetImages"
output_folder = r"D:\college\sem-8\dataset\cleaned_pet_dataset"

# ✅ CALL THE CLEANING FUNCTION
clean_and_save_dataset(input_folder, output_folder)


Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\10125.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\10501.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\10820.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\11095.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\11210.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\11565.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\11874.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\11935.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\12080.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\140.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\2663.jpg
Converting 1-channel to RGB: D:\college\sem-8\dataset\PetImages\Cat\2939.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\3300.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Cat\3491.jpg
Fixin

c:\Users\Ishan\Documents\sample_p1\env\Lib\site-packages\PIL\TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Fixing mode: CMYK -> RGB D:\college\sem-8\dataset\PetImages\Dog\9078.jpg
Fixing mode: P -> RGB D:\college\sem-8\dataset\PetImages\Dog\9188.jpg
✔ Finished cleaning.
✔ Saved images: 24998
✘ Skipped corrupted images: 0


# **Asthma FNN Model**

In [3]:
!pip install pandas --upgrade --force-reinstall


  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --------- ------------------------------ 2.6/11.0 MB 21.6 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.0 MB 13.4 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.0 MB 13.4 MB/s eta 0:00:01
   ----------------- ---------------------- 4.7/11.0 MB 6.8 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.0 MB 5.9 MB/s eta 0:00:01
   ---------------------------- ----------- 7.9/11.0 MB 6.3 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 6.2 MB/s eta 0:00:01
   ---------------------------------- ----- 9.4/11.0 MB 5.8 MB/s eta 0:00:01
   ----------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.14.1 requires numpy<2.3,>=1.23.5, but you have numpy 2.3.5 which is incompatible.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.5 which is incompatible.
tensorflow-model-optimization 0.8.0 requires numpy~=1.23, but you have numpy 2.3.5 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# ============================================
# 1. Load dataset
# ============================================
csv_path = r"D:\college\sem-8\dataset\asthma_disease_data (1).csv"   # change this!
df = pd.read_csv(csv_path)


print("[INFO] Columns detected:", df.columns.tolist())

# ============================================
# 2. Remove useless columns
# ============================================
drop_cols = ["PatientID", "DoctorInCharge", "Unnamed: 27"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# ============================================
# 3. Separate features and label
# ============================================
X = df.drop("Diagnosis", axis=1)
y = df["Diagnosis"]           # contains 0 or 1

# ============================================
# 4. Encode categorical columns
# ============================================
cat_cols = X.select_dtypes(include=["object"]).columns

X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

# ============================================
# 5. Train-test split
# ============================================
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ============================================
# 6. Scale numeric values
# ============================================
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

input_dim = X_train.shape[1]
print(f"[INFO] Input features: {input_dim}")

# ============================================
# 7. Build FNN model (Binary)
# ============================================
model = Sequential([
    Dense(128, activation="relu", input_shape=(input_dim,)),
    Dropout(0.3),

    Dense(64, activation="relu"),
    Dropout(0.3),

    Dense(32, activation="relu"),
    Dropout(0.2),

    Dense(1, activation="sigmoid")      # Binary output
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ============================================
# 8. Train model
# ============================================
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=32
)

# ============================================
# 9. Save model
# ============================================
model.save("asthma_fnn_binary.h5")
print("\nModel saved as asthma_fnn_binary.h5")


AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)